# GA4 Sequential Funnel Analysis

In [ ]:
WITH raw AS (
  SELECT
    user_pseudo_id,
    (SELECT value.int_value FROM UNNEST(event_params) WHERE key = 'ga_session_id') AS ga_session_id,
    event_name,
    event_timestamp
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20201130'
),

sessions_all AS (
  SELECT DISTINCT
    CONCAT(user_pseudo_id, '-', CAST(ga_session_id AS STRING)) AS session_key
  FROM raw
  WHERE ga_session_id IS NOT NULL
),

funnel_events AS (
  SELECT
    CONCAT(user_pseudo_id, '-', CAST(ga_session_id AS STRING)) AS session_key,
    event_name,
    event_timestamp
  FROM raw
  WHERE ga_session_id IS NOT NULL
    AND event_name IN ('view_item', 'add_to_cart', 'begin_checkout', 'purchase')
),

session_steps AS (
  SELECT
    session_key,
    MIN(IF(event_name = 'view_item', event_timestamp, NULL)) AS ts_view,
    MIN(IF(event_name = 'add_to_cart', event_timestamp, NULL)) AS ts_cart,
    MIN(IF(event_name = 'begin_checkout', event_timestamp, NULL)) AS ts_checkout,
    MIN(IF(event_name = 'purchase', event_timestamp, NULL)) AS ts_purchase
  FROM funnel_events
  GROUP BY session_key
),

agg AS (
  SELECT
    (SELECT COUNT(*) FROM sessions_all) AS total_sessions,

    COUNT(*) AS sessions_with_any_funnel_event,
    COUNTIF(ts_view IS NOT NULL) AS sessions_view,
    COUNTIF(ts_cart IS NOT NULL) AS sessions_cart,
    COUNTIF(ts_checkout IS NOT NULL) AS sessions_checkout,
    COUNTIF(ts_purchase IS NOT NULL) AS sessions_purchase,

    COUNTIF(ts_view IS NOT NULL AND ts_cart IS NOT NULL AND ts_cart >= ts_view) AS seq_view_to_cart,
    COUNTIF(ts_cart IS NOT NULL AND ts_checkout IS NOT NULL AND ts_checkout >= ts_cart) AS seq_cart_to_checkout,
    COUNTIF(ts_checkout IS NOT NULL AND ts_purchase IS NOT NULL AND ts_purchase >= ts_checkout) AS seq_checkout_to_purchase,

    COUNTIF(ts_checkout IS NOT NULL AND ts_cart IS NULL) AS checkout_without_cart,
    COUNTIF(ts_purchase IS NOT NULL AND ts_checkout IS NULL) AS purchase_without_checkout,
    COUNTIF(ts_purchase IS NOT NULL AND ts_cart IS NULL) AS purchase_without_cart
  FROM session_steps
)

SELECT
  total_sessions,
  sessions_with_any_funnel_event,

  sessions_view,
  sessions_cart,
  sessions_checkout,
  sessions_purchase,

  ROUND(100 * SAFE_DIVIDE(sessions_view, total_sessions), 2) AS reach_view_pct,
  ROUND(100 * SAFE_DIVIDE(sessions_cart, total_sessions), 2) AS reach_cart_pct,
  ROUND(100 * SAFE_DIVIDE(sessions_checkout, total_sessions), 2) AS reach_checkout_pct,
  ROUND(100 * SAFE_DIVIDE(sessions_purchase, total_sessions), 2) AS reach_purchase_pct,

  seq_view_to_cart,
  seq_cart_to_checkout,
  seq_checkout_to_purchase,

  ROUND(100 * SAFE_DIVIDE(seq_view_to_cart, sessions_view), 2) AS step_conv_view_to_cart_pct,
  ROUND(100 * SAFE_DIVIDE(seq_cart_to_checkout, sessions_cart), 2) AS step_conv_cart_to_checkout_pct,
  ROUND(100 * SAFE_DIVIDE(seq_checkout_to_purchase, sessions_checkout), 2) AS step_conv_checkout_to_purchase_pct,

  ROUND(100 * (1 - SAFE_DIVIDE(seq_view_to_cart, sessions_view)), 2) AS leak_after_view_pct,
  ROUND(100 * (1 - SAFE_DIVIDE(seq_cart_to_checkout, sessions_cart)), 2) AS leak_after_cart_pct,
  ROUND(100 * (1 - SAFE_DIVIDE(seq_checkout_to_purchase, sessions_checkout)), 2) AS leak_after_checkout_pct,

  checkout_without_cart,
  purchase_without_checkout,
  purchase_without_cart,

  ROUND(100 * SAFE_DIVIDE(checkout_without_cart, sessions_checkout), 2) AS checkout_without_cart_pct,
  ROUND(100 * SAFE_DIVIDE(purchase_without_checkout, sessions_purchase), 2) AS purchase_without_checkout_pct,
  ROUND(100 * SAFE_DIVIDE(purchase_without_cart, sessions_purchase), 2) AS purchase_without_cart_pct
FROM agg;



# Sequential Funnel Leakage (Session-Level)

## 1) What this analysis is doing

This query rebuilds sessions from event-level GA4 data and evaluates funnel progression **in timestamp order**.

* Session key: `user_pseudo_id + ga_session_id`
* Funnel events used: `view_item → add_to_cart → begin_checkout → purchase`
* For each session, we take first occurrence timestamp of each step
* A step transition is counted only when order is correct (e.g., `cart_ts >= view_ts`)

This moves us from “reach-only” funnel to **ordered progression**.

---

## 2) Key output snapshot

| Metric                         |           Value |
| ------------------------------ | --------------: |
| Total sessions                 |         108,401 |
| Sessions with any funnel event |          26,006 |
| Sessions with `view_item`      | 25,943 (23.93%) |
| Sessions with `add_to_cart`    |   2,201 (2.03%) |
| Sessions with `begin_checkout` |   4,585 (4.23%) |
| Sessions with `purchase`       |   1,617 (1.49%) |

### Ordered transition metrics

| Transition            | Sessions | Step Conversion | Leakage |
| --------------------- | -------: | --------------: | ------: |
| `view → cart`         |    2,193 |           8.45% |  91.55% |
| `cart → checkout`     |      849 |          38.57% |  61.43% |
| `checkout → purchase` |    1,615 |          35.22% |  64.78% |

### Path anomaly metrics

| Metric                    |                               Value |
| ------------------------- | ----------------------------------: |
| Checkout without cart     | 3,542 (77.25% of checkout sessions) |
| Purchase without checkout |      2 (0.12% of purchase sessions) |
| Purchase without cart     | 1,188 (73.47% of purchase sessions) |

---

## 3) Insight 1 — The largest leak is pre-commit (`view → cart`)

The biggest loss is still at the product-to-intent handoff:

* 25,943 sessions view items
* only 2,193 move to cart in correct order
* **91.55% leak after product view**

Interpretation:

* The core problem is not final payment completion.
* The core problem is converting browsing behavior into any commitment action.

---

## 4) Insight 2 — Cart is not the dominant path in this dataset

Cart appears to be a minority path:

* 77.25% of checkout sessions happen without cart
* 73.47% of purchase sessions happen without cart

Interpretation:

* Either users often take a direct-to-checkout path,
* or `add_to_cart` is under-captured for some flows,
* or both.

Important: this is a **strong pattern**, but still a pattern — not proof of instrumentation bug by itself.


## Insight 3: The Real Problem is View → ANY Commitment
**Look at the reach numbers:**

- 23.93% of sessions view items (browsing)
- Only 2.03% add to cart
- But 4.23% reach checkout (more than cart!)

**What's happening:**

1. 23.93% browse (view products)
2. ~21.9% drop off completely (91.55% leak)
3. ~4.23% reach checkout, while only 2.03% reach cart

The real funnel:

```text
Browse (23.93%)
    ↓
    ├─→ Add to cart (2.03%) → Checkout (partial) → Purchase (partial)
    └─→ Direct to checkout (1.2%+) → Purchase (majority)
The problem isn't the funnel steps - it's converting browsers into ANY form of commitment.
```

---

## 6) Behavior model

```text
All sessions: 108,401
└── Viewed item: 25,943 (23.93%)
    ├── Ordered view→cart: 2,193 (8.45% of viewers)
    │   └── Ordered cart→checkout: 849 (38.57% of cart sessions)
    └── Checkout sessions: 4,585 (4.23% of all sessions)
        ├── Checkout without cart: 3,542 (77.25%)
        └── Ordered checkout→purchase: 1,615

Purchase sessions: 1,617 (1.49%)
├── Without cart: 1,188 (73.47%)
└── Without checkout: 2 (0.12%)


## What is likely true (still hypothesis)

### Scenario A — Fast-path purchasing exists

**Why this is plausible (evidence):**

* `checkout_without_cart = 3,542`, which is **77.25%** of all checkout sessions (`4,585`).
* `purchase_without_cart = 1,188`, which is **73.47%** of all purchase sessions (`1,617`).
* `purchase_without_checkout = 2`, only **0.12%** of purchase sessions.

**Interpretation:**
Most purchase journeys still pass through checkout, but many do not pass through cart. That pattern is consistent with a fast path (cart bypass), even though the exact mechanism still needs validation.

---

### Scenario B — “Browse mode” and “buy mode” may be different behaviors

**Why this is plausible (evidence):**

* `sessions_view = 25,943` (**23.93%** of all sessions).
* Ordered `view → cart = 2,193`, so step conversion is only **8.45%**.
* Leakage after view is **91.55%**.
* Final purchase reach is **1.49%** of sessions.

**Interpretation:**
A large group engages at product-view level but does not commit. A smaller high-intent group progresses and converts. This supports a two-mode behavior framing (explore vs. buy).

---

### Scenario C — Cart may be optional for many successful journeys

**Why this is plausible (evidence):**

* `sessions_cart = 2,201` (**2.03%** reach) vs `sessions_checkout = 4,585` (**4.23%** reach).
* Checkout reach is more than double cart reach.
* `73.47%` of purchase sessions happen without cart.

**Interpretation:**
Cart is clearly not a universal step in successful journeys for this dataset. It should be treated as one path, not the only path.

---

## What we can do with the insight

### Insight 1 — Use dual-path funnel reporting

Track both paths in parallel and report both every time:

1. **Cart-assisted path:** `view → cart → checkout → purchase`
2. **Non-cart path:** `view/entry → checkout → purchase`

**Why:**
Current counts show non-cart behavior is too large to hide inside a single linear funnel (`checkout_without_cart = 77.25%`, `purchase_without_cart = 73.47%`).

---

### Insight 2 — Prioritize browse-to-commit optimization

Focus optimization on the first commitment step, not only late checkout polish.

**Why:**
The largest loss is early (`view → cart` conversion **8.45%**, leakage **91.55%**).
This is where the biggest volume of recoverable users sits.

**Focus areas:**

* stronger product-page clarity
* clearer commitment CTAs
* lower friction before checkout intent

---

### Insight 3 — Treat cart metrics as conditional KPIs

Keep cart KPIs, but do not use cart as the universal denominator for conversion health.

**Why:**
Cart participation is low relative to checkout and purchase behavior (`2.03%` cart reach vs `4.23%` checkout reach; `73.47%` of purchases without cart).

**Reporting rule:**

* keep cart KPIs,
* add non-cart conversion KPIs,
* avoid cart-only funnel conclusions.


In [ ]:
WITH raw AS (
  SELECT
    user_pseudo_id,
    (SELECT value.int_value FROM UNNEST(event_params) WHERE CAST(key AS STRING) = 'ga_session_id') AS ga_session_id,
    event_name,
    event_timestamp,
    device.category AS device_category,
    geo.country AS country,
    LOWER(TRIM(traffic_source.source)) AS ft_source,
    LOWER(TRIM(traffic_source.medium)) AS ft_medium,
    user_first_touch_timestamp
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20201130'
),

session_rollup AS (
  SELECT
    CONCAT(user_pseudo_id, '-', CAST(ga_session_id AS STRING)) AS session_key,
    ARRAY_AGG(device_category ORDER BY event_timestamp LIMIT 1)[OFFSET(0)] AS device,
    ARRAY_AGG(country ORDER BY event_timestamp LIMIT 1)[OFFSET(0)] AS country,
    ARRAY_AGG(ft_source ORDER BY event_timestamp LIMIT 1)[OFFSET(0)] AS ft_source,
    ARRAY_AGG(ft_medium ORDER BY event_timestamp LIMIT 1)[OFFSET(0)] AS ft_medium,
    MIN(user_first_touch_timestamp) AS user_first_touch_ts,
    MIN(IF(event_name = 'view_item', event_timestamp, NULL)) AS ts_view,
    MIN(IF(event_name = 'add_to_cart', event_timestamp, NULL)) AS ts_cart,
    MIN(IF(event_name = 'begin_checkout', event_timestamp, NULL)) AS ts_checkout,
    MIN(IF(event_name = 'purchase', event_timestamp, NULL)) AS ts_purchase
  FROM raw
  WHERE ga_session_id IS NOT NULL
  GROUP BY session_key
),

flags AS (
  SELECT
    *,
    (ts_view IS NOT NULL) AS has_view,
    (ts_cart IS NOT NULL) AS has_cart,
    (ts_checkout IS NOT NULL) AS has_checkout,
    (ts_purchase IS NOT NULL) AS has_purchase,
    (ts_view IS NOT NULL AND ts_cart IS NOT NULL AND ts_cart >= ts_view) AS seq_view_to_cart,
    (ts_cart IS NOT NULL AND ts_checkout IS NOT NULL AND ts_checkout >= ts_cart) AS seq_cart_to_checkout,
    (ts_checkout IS NOT NULL AND ts_purchase IS NOT NULL AND ts_purchase >= ts_checkout) AS seq_checkout_to_purchase,
    (ts_checkout IS NOT NULL AND ts_cart IS NULL) AS checkout_without_cart,
    (ts_purchase IS NOT NULL AND ts_cart IS NULL) AS purchase_without_cart,
    CASE
      WHEN DATE(TIMESTAMP_MICROS(user_first_touch_ts)) BETWEEN DATE '2020-11-01' AND DATE '2020-11-30'
        THEN 'new_in_period'
      ELSE 'returning'
    END AS user_type,
    CASE
      WHEN ft_source IS NULL OR ft_source = '' OR ft_medium IS NULL OR ft_medium = ''
        THEN 'null_or_blank'
      WHEN ft_source IN ('<other>', '(data deleted)', '(not set)', 'not set', 'unknown', 'unassigned', '(unassigned)')
        OR ft_medium IN ('<other>', '(data deleted)', '(not set)', 'not set', 'unknown', 'unassigned', '(unassigned)')
        THEN 'placeholder_or_deleted'
      WHEN ft_source = '(direct)' OR ft_medium IN ('(none)', '(not set)', 'none', 'not set')
        THEN 'direct_or_none'
      WHEN ft_source = 'google' AND ft_medium = 'organic'
        THEN 'google_organic'
      WHEN ft_source = 'google' AND REGEXP_CONTAINS(ft_medium, r'cpc|ppc|paid')
        THEN 'google_paid'
      WHEN ft_medium = 'referral'
        THEN 'referral'
      ELSE 'other_attributed'
    END AS source_group
  FROM session_rollup
),

top_countries AS (
  SELECT country
  FROM flags
  WHERE country IS NOT NULL
  GROUP BY country
  ORDER BY COUNT(*) DESC
  LIMIT 10
),

segment_rows AS (
  SELECT 'device' AS segment_type, COALESCE(device, 'unknown') AS segment_value, * FROM flags
  UNION ALL
  SELECT 'country' AS segment_type, country AS segment_value, f.* FROM flags f JOIN top_countries c USING (country)
  UNION ALL
  SELECT 'user_type' AS segment_type, user_type AS segment_value, * FROM flags
  UNION ALL
  SELECT 'source_group' AS segment_type, source_group AS segment_value, * FROM flags
),

agg AS (
  SELECT
    segment_type,
    segment_value,
    COUNT(*) AS total_sessions,
    COUNTIF(has_view) AS sessions_view,
    COUNTIF(has_cart) AS sessions_cart,
    COUNTIF(has_checkout) AS sessions_checkout,
    COUNTIF(has_purchase) AS sessions_purchase,
    COUNTIF(seq_view_to_cart) AS seq_view_to_cart_sessions,
    COUNTIF(seq_cart_to_checkout) AS seq_cart_to_checkout_sessions,
    COUNTIF(seq_checkout_to_purchase) AS seq_checkout_to_purchase_sessions,
    COUNTIF(has_view AND NOT seq_view_to_cart) AS view_leak_sessions,
    COUNTIF(has_cart AND NOT seq_cart_to_checkout) AS cart_leak_sessions,
    COUNTIF(has_checkout AND NOT seq_checkout_to_purchase) AS checkout_leak_sessions,
    COUNTIF(checkout_without_cart) AS checkout_without_cart_sessions,
    COUNTIF(purchase_without_cart) AS purchase_without_cart_sessions
  FROM segment_rows
  GROUP BY segment_type, segment_value
),

totals AS (
  SELECT COUNTIF(has_view AND NOT seq_view_to_cart) AS total_view_leak_sessions
  FROM flags
),

final AS (
  SELECT
    a.segment_type,
    a.segment_value,
    a.total_sessions,
    a.sessions_view,
    a.sessions_cart,
    a.sessions_checkout,
    a.sessions_purchase,
    ROUND(100 * SAFE_DIVIDE(a.sessions_view, a.total_sessions), 2) AS reach_view_pct,
    ROUND(100 * SAFE_DIVIDE(a.sessions_cart, a.total_sessions), 2) AS reach_cart_pct,
    ROUND(100 * SAFE_DIVIDE(a.sessions_checkout, a.total_sessions), 2) AS reach_checkout_pct,
    ROUND(100 * SAFE_DIVIDE(a.sessions_purchase, a.total_sessions), 2) AS reach_purchase_pct,
    ROUND(100 * SAFE_DIVIDE(a.seq_view_to_cart_sessions, a.sessions_view), 2) AS step_conv_view_to_cart_pct,
    ROUND(100 * SAFE_DIVIDE(a.seq_cart_to_checkout_sessions, a.sessions_cart), 2) AS step_conv_cart_to_checkout_pct,
    ROUND(100 * SAFE_DIVIDE(a.seq_checkout_to_purchase_sessions, a.sessions_checkout), 2) AS step_conv_checkout_to_purchase_pct,
    ROUND(100 * SAFE_DIVIDE(a.view_leak_sessions, a.sessions_view), 2) AS leak_after_view_pct,
    ROUND(100 * SAFE_DIVIDE(a.cart_leak_sessions, a.sessions_cart), 2) AS leak_after_cart_pct,
    ROUND(100 * SAFE_DIVIDE(a.checkout_leak_sessions, a.sessions_checkout), 2) AS leak_after_checkout_pct,
    ROUND(100 * SAFE_DIVIDE(a.checkout_without_cart_sessions, a.sessions_checkout), 2) AS checkout_without_cart_pct,
    ROUND(100 * SAFE_DIVIDE(a.purchase_without_cart_sessions, a.sessions_purchase), 2) AS purchase_without_cart_pct,
    a.view_leak_sessions,
    ROUND(100 * SAFE_DIVIDE(a.view_leak_sessions, t.total_view_leak_sessions), 2) AS share_of_total_view_leak_pct
  FROM agg a
  CROSS JOIN totals t
)

SELECT *
FROM final
ORDER BY segment_type, leak_after_view_pct DESC, total_sessions DESC;


## Query L2 — Sequential Funnel Leakage by Segment

### Objective

Decompose funnel leakage by major segments to answer two questions:

1. **Where is leakage rate worst?** (efficiency problem)
2. **Where is leakage volume largest?** (impact problem)

This avoids optimizing a small high-leak segment while ignoring large-volume leak drivers.

---

### What the query is doing

1. Rebuilds session-level funnel steps using:

   * `session_key = user_pseudo_id || '-' || ga_session_id`
   * first timestamp per step: `view_item`, `add_to_cart`, `begin_checkout`, `purchase`
2. Computes ordered transitions:

   * `view → cart`
   * `cart → checkout`
   * `checkout → purchase`
3. Produces metrics by segment:

   * reach, step conversion, leakage, cart-bypass behavior
4. Adds **volume share of leakage**:

   * `share_of_total_view_leak_pct` = segment’s contribution to total `view`-stage leakage

---

### Important definitions in this query

#### New vs returning users

* **new_in_period**: `DATE(user_first_touch_timestamp)` is between `2020-11-01` and `2020-11-30`
* **returning**: first touch occurred **before** Nov 2020

This is period-based cohorting, which is appropriate for month-scope analysis.

#### Source group

Source groups are built from **first-touch** `traffic_source.source/medium` into coarse buckets:

* `google_organic`, `google_paid`, `referral`, `direct_or_none`, `placeholder_or_deleted`, etc.

Given known obfuscation, this should be interpreted as directional segmentation, not precise attribution.

---

## Output summary and interpretation

### 1) Global pattern is stable across segments

* `leak_after_view_pct` is consistently high across segments (~**91–94%**).
* Core bottleneck remains unchanged: **view → commitment**.

**Meaning:** segmentation does not change the main diagnosis; it helps prioritize where to act first.

---

### 2) User-type split is the most actionable finding

| Metric                       | Returning | New in period |
| ---------------------------- | --------: | ------------: |
| Reach: view                  |    33.05% |        22.47% |
| Reach: checkout              |     6.71% |         3.83% |
| Reach: purchase              |     3.83% |         1.12% |
| Step conv: view→cart         |     6.51% |         8.91% |
| Step conv: cart→checkout     |    51.08% |        36.42% |
| Step conv: checkout→purchase |    57.11% |        29.08% |
| Checkout without cart        |    83.08% |        75.61% |
| Purchase without cart        |    81.53% |        69.03% |
| Share of total view leakage  |    19.49% |        80.51% |

**Readout**

* Returning users show stronger late-stage conversion once they enter checkout.
* Returning users bypass cart even more heavily.
* New users drive most absolute leakage volume (**80.51%**) simply due to much larger base.

**Implication**

* Acquisition/first-session UX should target browse-to-commit conversion.
* Returning-user experience should prioritize fast checkout path quality.

---

### 3) Device differences are smaller than expected

| Metric                       | Mobile | Desktop | Tablet |
| ---------------------------- | -----: | ------: | -----: |
| Step conv: view→cart         |  8.36% |   8.52% |  8.36% |
| Leak after view              | 91.64% |  91.48% | 91.64% |
| Step conv: checkout→purchase | 36.31% |  34.55% | 32.65% |
| Share of total view leakage  | 39.95% |  57.84% |  2.21% |

**Readout**

* Early-stage leak is similarly high on mobile and desktop.
* Desktop contributes more leakage volume because of larger traffic share.
* Mobile has slightly better checkout→purchase performance in this slice.

**Implication**

* This is not a mobile-only issue; prioritize by volume (desktop + mobile), not by assumption.

---

### 4) Country pattern: rate variation exists, but volume is concentrated

* US contributes **43.88%** of total view leakage (largest impact segment).
* India contributes **9.63%**, Canada **7.63%**.
* Leak rates are high across all top countries (roughly 90%+), with smaller countries showing more volatility due to sample size.

**Implication**

* Prioritize US first for highest impact.
* Treat smaller-country rate differences as secondary until stabilized over longer window.

---

### 5) Source-group readout (coarse, with caveat)

* Largest leakage volume contributions:

  * `placeholder_or_deleted`: **33.55%**
  * `google_organic`: **30.51%**
  * `direct_or_none`: **22.93%**
* `referral` shows stronger late-stage conversion (`checkout→purchase = 44.32%`) in this slice.

**Interpretation caution**

* Because source quality is mixed, use this for broad prioritization only.
* Do not make narrow channel-level claims from this dataset.



In [ ]:
-- L3: Path economics by session path family
WITH raw AS (
  SELECT
    user_pseudo_id,
    (SELECT value.int_value FROM UNNEST(event_params) WHERE CAST(key AS STRING) = 'ga_session_id') AS ga_session_id,
    event_name,
    event_timestamp,
    device.category AS device_category,
    user_first_touch_timestamp,
    ecommerce.transaction_id AS transaction_id,
    ecommerce.purchase_revenue AS purchase_revenue,
    ecommerce.purchase_revenue_in_usd AS purchase_revenue_in_usd
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20201130'
),

sessions AS (
  SELECT
    CONCAT(user_pseudo_id, '-', CAST(ga_session_id AS STRING)) AS session_key,
    ARRAY_AGG(device_category ORDER BY event_timestamp LIMIT 1)[OFFSET(0)] AS device,
    MIN(user_first_touch_timestamp) AS user_first_touch_ts
  FROM raw
  WHERE ga_session_id IS NOT NULL
  GROUP BY 1
),

steps AS (
  SELECT
    CONCAT(user_pseudo_id, '-', CAST(ga_session_id AS STRING)) AS session_key,
    MIN(IF(event_name = 'view_item', event_timestamp, NULL)) AS ts_view,
    MIN(IF(event_name = 'add_to_cart', event_timestamp, NULL)) AS ts_cart,
    MIN(IF(event_name = 'begin_checkout', event_timestamp, NULL)) AS ts_checkout,
    MIN(IF(event_name = 'purchase', event_timestamp, NULL)) AS ts_purchase
  FROM raw
  WHERE ga_session_id IS NOT NULL
  GROUP BY 1
),

-- Event-level purchase revenue by session (diagnostic reference)
purchase_events_by_session AS (
  SELECT
    CONCAT(user_pseudo_id, '-', CAST(ga_session_id AS STRING)) AS session_key,
    SUM(
      IF(
        event_name = 'purchase',
        COALESCE(purchase_revenue_in_usd, purchase_revenue),
        0
      )
    ) AS revenue_event_level
  FROM raw
  WHERE ga_session_id IS NOT NULL
  GROUP BY 1
),

-- Deduplicate transaction revenue at (session_key, transaction_id)
tx_dedup AS (
  SELECT
    CONCAT(user_pseudo_id, '-', CAST(ga_session_id AS STRING)) AS session_key,
    transaction_id,
    MAX(COALESCE(purchase_revenue_in_usd, purchase_revenue)) AS tx_revenue
  FROM raw
  WHERE ga_session_id IS NOT NULL
    AND event_name = 'purchase'
    AND transaction_id IS NOT NULL
  GROUP BY 1, 2
),

tx_by_session AS (
  SELECT
    session_key,
    COUNT(*) AS transactions,
    SUM(tx_revenue) AS revenue_txn_dedup
  FROM tx_dedup
  GROUP BY 1
),

session_facts AS (
  SELECT
    s.session_key,
    s.device,
    CASE
      WHEN DATE(TIMESTAMP_MICROS(s.user_first_touch_ts)) BETWEEN DATE '2020-11-01' AND DATE '2020-11-30'
        THEN 'new_in_period'
      ELSE 'returning'
    END AS user_type,

    st.ts_view,
    st.ts_cart,
    st.ts_checkout,
    st.ts_purchase,

    COALESCE(t.transactions, 0) AS transactions,
    COALESCE(t.revenue_txn_dedup, 0) AS revenue_txn_dedup,
    COALESCE(pe.revenue_event_level, 0) AS revenue_event_level
  FROM sessions s
  LEFT JOIN steps st USING (session_key)
  LEFT JOIN tx_by_session t USING (session_key)
  LEFT JOIN purchase_events_by_session pe USING (session_key)
),

labeled AS (
  SELECT
    *,
    (ts_view IS NOT NULL) AS has_view,
    (ts_cart IS NOT NULL) AS has_cart,
    (ts_checkout IS NOT NULL) AS has_checkout,
    (ts_purchase IS NOT NULL) AS has_purchase,

    CASE
      WHEN ts_purchase IS NOT NULL
        AND ts_checkout IS NOT NULL
        AND ts_checkout <= ts_purchase
        AND ts_cart IS NOT NULL
        AND ts_cart <= ts_checkout
        THEN 'purchase_cart_assisted'

      WHEN ts_purchase IS NOT NULL
        AND ts_checkout IS NOT NULL
        AND ts_checkout <= ts_purchase
        AND (ts_cart IS NULL OR ts_cart > ts_checkout)
        THEN 'purchase_non_cart'

      WHEN ts_checkout IS NOT NULL
        AND ts_purchase IS NULL
        AND ts_cart IS NOT NULL
        AND ts_cart <= ts_checkout
        THEN 'checkout_no_purchase_cart_assisted'

      WHEN ts_checkout IS NOT NULL
        AND ts_purchase IS NULL
        AND (ts_cart IS NULL OR ts_cart > ts_checkout)
        THEN 'checkout_no_purchase_non_cart'

      WHEN ts_cart IS NOT NULL
        AND ts_checkout IS NULL
        AND ts_purchase IS NULL
        THEN 'cart_no_checkout'

      WHEN ts_view IS NOT NULL
        AND ts_checkout IS NULL
        AND ts_purchase IS NULL
        THEN 'view_only'

      ELSE 'other_or_no_funnel'
    END AS path_family
  FROM session_facts
),

agg AS (
  SELECT
    path_family,
    COUNT(*) AS sessions,
    COUNTIF(has_purchase) AS purchase_sessions,
    SUM(transactions) AS transactions,
    SUM(revenue_txn_dedup) AS revenue_txn_dedup,
    SUM(revenue_event_level) AS revenue_event_level,
    AVG(CASE WHEN has_purchase THEN revenue_txn_dedup ELSE NULL END) AS avg_rev_per_purchase_session
  FROM labeled
  GROUP BY 1
)

SELECT
  path_family,

  sessions,
  ROUND(100 * SAFE_DIVIDE(sessions, SUM(sessions) OVER()), 2) AS session_share_pct,

  purchase_sessions,
  ROUND(100 * SAFE_DIVIDE(purchase_sessions, sessions), 2) AS purchase_rate_pct,

  transactions,
  ROUND(100 * SAFE_DIVIDE(transactions, SUM(transactions) OVER()), 2) AS transaction_share_pct,

  ROUND(revenue_txn_dedup, 2) AS revenue_txn_dedup,
  ROUND(100 * SAFE_DIVIDE(revenue_txn_dedup, SUM(revenue_txn_dedup) OVER()), 2) AS revenue_share_pct,

  ROUND(SAFE_DIVIDE(revenue_txn_dedup, NULLIF(transactions, 0)), 2) AS aov_txn_dedup,
  ROUND(avg_rev_per_purchase_session, 2) AS avg_rev_per_purchase_session,

  -- Diagnostic reference (can be higher if purchase events are duplicated)
  ROUND(revenue_event_level, 2) AS revenue_event_level
FROM agg
ORDER BY revenue_txn_dedup DESC, sessions DESC;


## Query L3 — Path Economics (Cart-Assisted vs Non-Cart)

### Objective

Turn funnel behavior into business value:

* Which path drives more **purchases / transactions / revenue**
* Which path is more **efficient**
* Which path carries more **economic value per order/session**


## What this query does (step by step)

### Step 1) Read event-level data for Nov 2020

Pulls session identifiers, funnel events, timestamps, device/user metadata, and purchase fields (`transaction_id`, revenue).

### Step 2) Rebuild sessions

Creates:

* `session_key = user_pseudo_id || '-' || ga_session_id`
* one row per session with session-level attributes

### Step 3) Build ordered funnel timestamps per session

For each session:

* first `view_item` timestamp
* first `add_to_cart` timestamp
* first `begin_checkout` timestamp
* first `purchase` timestamp

This lets path logic use actual event order, not just event presence.

### Step 4) Build revenue in two ways

* **`revenue_event_level`**: raw sum of purchase-event revenue (can include duplicate purchase fires)
* **`revenue_txn_dedup`**: deduped at `(session_key, transaction_id)` then summed
  (more conservative for economics)

### Step 5) Label each session into one path family

Main families:

* `purchase_non_cart`
* `purchase_cart_assisted`
* `checkout_no_purchase_non_cart`
* `checkout_no_purchase_cart_assisted`
* `cart_no_checkout`
* `view_only`
* `other_or_no_funnel`

### Step 6) Aggregate economics by path

Computes:

* session share
* purchase sessions
* transaction share
* revenue share
* AOV (`revenue_txn_dedup / transactions`)
* avg revenue per purchase session


## Output (key rows)

| path_family                        | sessions | session_share_pct | transactions | transaction_share_pct | revenue_txn_dedup | revenue_share_pct | aov_txn_dedup |
| ---------------------------------- | -------: | ----------------: | -----------: | --------------------: | ----------------: | ----------------: | ------------: |
| purchase_non_cart                  |    1,198 |             1.11% |        1,240 |                73.20% |            86,029 |            71.08% |         69.38 |
| purchase_cart_assisted             |      417 |             0.38% |          452 |                26.68% |            35,005 |            28.92% |         77.44 |
| checkout_no_purchase_non_cart      |    2,538 |             2.34% |            0 |                 0.00% |                 0 |             0.00% |          null |
| checkout_no_purchase_cart_assisted |      432 |             0.40% |            0 |                 0.00% |                 0 |             0.00% |          null |
| view_only                          |   20,261 |            18.69% |            0 |                 0.00% |                 0 |             0.00% |          null |
| cart_no_checkout                   |    1,158 |             1.07% |            0 |                 0.00% |                 0 |             0.00% |          null |
| other_or_no_funnel                 |   82,397 |            76.01% |            2 |                 0.12% |                 0 |             0.00% |             0 |


## Interpretation

### 1) Non-cart purchase path is the dominant revenue engine

* `purchase_non_cart` drives **73.20% of transactions** and **71.08% of deduped revenue**.
* `purchase_cart_assisted` drives **26.68% of transactions** and **28.92% of deduped revenue**.

**Meaning:**
Most monetization comes from sessions that purchase without cart as a valid prior step.



### 2) Cart-assisted path is smaller, but higher value per order

* AOV:

  * `purchase_cart_assisted`: **77.44**
  * `purchase_non_cart`: **69.38**
* Avg revenue per purchase session:

  * cart-assisted: **83.94**
  * non-cart: **71.81**

**Meaning:**
Cart-assisted buyers are fewer, but basket value is higher.
So this is a **volume vs value** split, not a winner-takes-all path decision.


### 3) Biggest near-term loss is non-cart checkout abandonment

* `checkout_no_purchase_non_cart`: **2,538 sessions**
* `checkout_no_purchase_cart_assisted`: **432 sessions**

Non-cart checkout abandonment is ~**85.45%** of checkout-no-purchase sessions (`2,538 / (2,538+432)`).

**Meaning:**
If optimizing checkout completion, non-cart checkout flow is the highest-impact target.


### 4) Large “other_or_no_funnel” share is expected at session level

* `other_or_no_funnel` = **76.01% of all sessions**

**Meaning:**
Most sessions do not enter the core ecommerce funnel events (`view_item/cart/checkout/purchase`) and are browsing/navigation-heavy. This is normal in event-level ecommerce traffic.


## Caveats to keep explicit

1. `purchase_rate_pct = 100%` for purchase-labeled families is by construction (label definition), not a performance rate.
2. `revenue_event_level` is much higher than `revenue_txn_dedup` for non-cart path (108,696 vs 86,029), indicating potential duplicate purchase-event firing; deduped revenue is the safer economics view.
3. Transaction IDs appear reused/duplicated enough that economics should be presented with the dedup method clearly documented.
